# Kernel methods 

### Import des fonctions


In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.linalg import solve,lstsq
from scipy.stats import uniform
from cvxopt import matrix
from cvxopt import solvers
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from itertools import compress
import numexpr as ne

In [5]:
X_0=pd.read_csv("Data/Xtr0_mat100.csv",sep=' ',header=None)
X_1=pd.read_csv("Data/Xtr1_mat100.csv",sep=' ',header=None)
X_2=pd.read_csv("Data/Xtr2_mat100.csv",sep=' ',header=None)


Y_0=pd.read_csv("Data/Ytr0.csv",sep=',')
Y_1=pd.read_csv("Data/Ytr1.csv",sep=',')
Y_2=pd.read_csv("Data/Ytr2.csv",sep=',')

X_0

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.010870,0.010870,0.043478,0.010870,0.021739,0.021739,0.000000,0.010870,0.010870,0.010870,...,0.000000,0.032609,0.01087,0.010870,0.010870,0.000000,0.000000,0.010870,0.000000,0.01087
1,0.000000,0.000000,0.010870,0.010870,0.000000,0.010870,0.010870,0.000000,0.000000,0.000000,...,0.010870,0.010870,0.01087,0.021739,0.000000,0.010870,0.021739,0.032609,0.000000,0.00000
2,0.021739,0.010870,0.021739,0.000000,0.000000,0.000000,0.010870,0.010870,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.010870,0.000000,0.021739,0.000000,0.021739,0.021739,0.01087
3,0.021739,0.010870,0.043478,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.010870,...,0.021739,0.000000,0.00000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.01087
4,0.021739,0.010870,0.010870,0.010870,0.010870,0.010870,0.032609,0.000000,0.010870,0.043478,...,0.010870,0.010870,0.00000,0.000000,0.010870,0.010870,0.032609,0.010870,0.021739,0.01087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000000,0.000000,0.000000,0.010870,0.000000,0.010870,0.000000,0.021739,0.010870,0.032609,...,0.000000,0.010870,0.01087,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.00000
1996,0.010870,0.021739,0.000000,0.000000,0.000000,0.021739,0.010870,0.010870,0.021739,0.000000,...,0.021739,0.010870,0.00000,0.010870,0.000000,0.000000,0.000000,0.000000,0.010870,0.01087
1997,0.010870,0.000000,0.000000,0.000000,0.000000,0.010870,0.010870,0.010870,0.021739,0.000000,...,0.010870,0.010870,0.00000,0.000000,0.021739,0.000000,0.032609,0.032609,0.000000,0.00000
1998,0.010870,0.010870,0.000000,0.000000,0.021739,0.000000,0.010870,0.021739,0.000000,0.032609,...,0.032609,0.000000,0.00000,0.010870,0.000000,0.010870,0.000000,0.000000,0.000000,0.01087


In [6]:
print(Y_0["Bound"].value_counts())
print(Y_1["Bound"].value_counts())
print(Y_2["Bound"].value_counts())

0    1038
1     962
Name: Bound, dtype: int64
1    1001
0     999
Name: Bound, dtype: int64
0    1003
1     997
Name: Bound, dtype: int64


Les classes sont plutôt équilibrées 

In [7]:
Y_0train=np.where(Y_0["Bound"]==0,-1,Y_0["Bound"])
Y_1train=np.where(Y_1["Bound"]==0,-1,Y_1["Bound"])
Y_2train=np.where(Y_2["Bound"]==0,-1,Y_2["Bound"])

X_0train=X_0.to_numpy()
X_1train=X_1.to_numpy()
X_2train=X_2.to_numpy()
print(X_0train.shape)

(2000, 100)


## Ridge regression ( équivalent à Kernel ridge régression avec noyau linéaire) 
On ajoute une colonne de biais 

In [8]:
X_0train_forRR=np.hstack((np.ones((X_0train.shape[0],1)),X_0train))
X_1train_forRR=np.hstack((np.ones((X_0train.shape[0],1)),X_0train))
X_2train_forRR=np.hstack((np.ones((X_0train.shape[0],1)),X_0train))

### Solver ridge Regression 
Même si les classes sont équilibrées je laisse la possibilité de modifier les poids. Enfin pas vraiment puisque le seul poids que je met quand on souhaite en mettre c'est le poid correspondant au poids des classes dans le set qui sera entraîné.
J'utilisais lstsq pour résoudre le système lineaire car plus stable si la matrice est mal conditionné/pas inversible mais cela prend beaucoup plus de temps que solve, ducoup je suis repassé à solve.

In [113]:
def Ridge_Regresssion(X_train,y_train,lbd,weight=False):
    n=y_train.shape[0]
    d=X_train.shape[1]
    if not(weight):
        A=(X_train.T.dot(X_train))/n+lbd*np.eye(d)
        B=X_train.T.dot(y_train)/n
        theta=solve(A,B)
        return theta
    else:
        w1=(y_train==1).mean()
        w0=1-w1
        w=np.where(y_train==1,w1,w0)
        W=np.diag(w)
        A=(X_train.T.dot(W.dot(X_train)))/n+lbd*np.eye(d)
        B=X_train.T.dot(W.dot(y_train))/n
        theta=solve(A,B)
        return theta


 ### Création de l'estimateur Ridge regression compatible avec l'API de scikit learn

In [17]:
class LinearRR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,weight=False):
        self.lbd=lbd
        self.weight=weight
        
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        
        self.theta=Ridge_Regresssion(X,y,self.lbd,self.weight)
        return self
    def decision_function(self,X):
        return X.dot(self.theta)
        
    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
   
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"weight":self.weight}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### Fonction permettant d'afficher les n meilleurs résultats avec les paramètres associées d'une grid search dans un tableau (dataframe)

In [9]:
def présentation_résultat(search,n):
    mask=search.cv_results_['rank_test_score']<=n
    params=list(compress(search.cv_results_['params'], list(mask)))
    mean_test_score=search.cv_results_['mean_test_score'][mask]
    a={}
    for i in range(mean_test_score.size):
        k=''
        for key, value in params[i].items():
            k+=" "+key+" "+str(value)
        a.update({k:mean_test_score[i]})
        sortedDict = sorted(a.items(), key=lambda x: x[1],reverse=True)
    l=[]
    for i in sortedDict:
        u=i[0].split(sep=' ')
        del(u[0])
        lp=[]
        for j in u[1::2]:
            lp.append(j)
        lp.append(i[1])
        l.append(lp)
    head=list(params[0].keys())+["mean_test_score"]

    return(pd.DataFrame(l,columns=head))
        
    
    
    
    

## Première recherche des hyperparamètres pour chaque train set 
Je crée un pipeline composé d'un transformer de données et de la ridge regression. Dans la manière dont j'ai défini la pipeline le transformer en question est *StandardScaler()* qui standardise les données ((X-mean)/std)). Attention cela standardise les données selon la mean et std du train set et donc lors d'une prédiction le test/validation set sera standardisé par rapport à la mean et std du training set. 
En soit la manière dont j'ai défini le tranformeur n'a pas d'importance car dans la gridSearch je teste d'autres transformeur à certain moment(mais fallait bien en mettre pour faire la pipeline), il y a même une option *"passthrough"* pour la première étape de la pipeline qui veut dire tout simplement passer à l'étape suivante et donc ne pas transfomrer mes données.


In [141]:
RR=Pipeline([("preprocess",StandardScaler()),("linearrr",LinearRR())])
print(RR)

hps0={"linearrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR0= GridSearchCV(RR,hps0,scoring="accuracy",cv=5,verbose=1)
searchLRR0.fit(X_0train_forRR, Y_0train)

hps1={"linearrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR1= GridSearchCV(RR,hps1,scoring="accuracy",cv=5,verbose=1)
searchLRR1.fit(X_1train_forRR, Y_1train)


hps2={"linearrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR2= GridSearchCV(RR,hps2,scoring="accuracy",cv=5,verbose=1)
searchLRR2.fit(X_2train_forRR, Y_2train)


Pipeline(steps=[('preprocess', StandardScaler()), ('linearrr', LinearRR())])
Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:   13.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:   13.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:   14.0s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess', StandardScaler()),
                                       ('linearrr', LinearRR())]),
             param_grid={'linearrr__lbd': [1e-06, 1e-05, 0.0001, 0.001, 0.01,
                                           0.1, 1, 10],
                         'linearrr__weight': [False, True],
                         'preprocess': ['passthrough', StandardScaler(),
                                        RobustScaler(), MinMaxScaler()]},
             scoring='accuracy', verbose=1)

## Affichage des résulats 

In [138]:
print(présentation_résultat(searchLRR0,10))


   linearrr__lbd linearrr__weight        preprocess  mean_test_score
0              1             True  StandardScaler()           0.5975
1              1            False  StandardScaler()           0.5945
2            0.1             True  StandardScaler()           0.5940
3           0.01            False    MinMaxScaler()           0.5930
4            0.1            False  StandardScaler()           0.5905
5          0.001             True    MinMaxScaler()           0.5890
6           0.01            False  StandardScaler()           0.5885
7           0.01             True  StandardScaler()           0.5870
8          0.001             True  StandardScaler()           0.5865
9          1e-06             True  StandardScaler()           0.5860
10         1e-05             True  StandardScaler()           0.5860
11        0.0001             True  StandardScaler()           0.5860
12         0.001            False  StandardScaler()           0.5860
   linearrr__lbd linearrr__weight 

In [ ]:
print(présentation_résultat(searchLRR1,10))

In [ ]:
print(présentation_résultat(searchLRR2,10))

### Recherche plus fine dans les zones d'intérêts déterminés à l'aide la search précedente

Ici je fais une RandomizedGrid search car j'ai beaucoup plus d'informations sur les zones d'intérêts où il faut chercher(notamment pour le lambda). Si je l'avais fait avant étant donné que j'avais aucun à priori sur la valeur de lambda la recherche au hasard aurait été plus difficile. 

In [144]:

hps0={"linearrr__lbd":uniform(loc=0.005,scale=5),"linearrr__weight":[False,True]}
searchLRR0= RandomizedSearchCV(RR,hps0,n_iter=1000,scoring="accuracy",cv=5,verbose=1)
searchLRR0.fit(X_0train_forRR, Y_0train)

hps1={"linearrr__lbd":uniform(loc=0.05,scale=10),"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR1= RandomizedSearchCV(RR,hps1,n_iter=1000,scoring="accuracy",cv=5,verbose=1)
searchLRR1.fit(X_1train_forRR, Y_1train)


hps2={"linearrr__lbd":uniform(loc=0.005,scale=10),"linearrr__weight":[False,True],
     "preprocess":["passthrough"]}
searchLRR2= RandomizedSearchCV(RR,hps2,n_iter=1000,scoring="accuracy",cv=5,verbose=1)
searchLRR2.fit(X_2train_forRR, Y_2train)


Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 5000 out of 5000 | elapsed:  2.7min finished


Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 5000 out of 5000 | elapsed:  3.6min finished


Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 5000 out of 5000 | elapsed:  1.8min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocess', StandardScaler()),
                                             ('linearrr', LinearRR())]),
                   n_iter=1000,
                   param_distributions={'linearrr__lbd': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000258DC893B50>,
                                        'linearrr__weight': [False, True],
                                        'preprocess': ['passthrough']},
                   scoring='accuracy', verbose=1)

### Présentation des résultats 

In [149]:
print(présentation_résultat(searchLRR0,10))


         linearrr__lbd linearrr__weight  mean_test_score
0   0.6808368789557301             True           0.5995
1   0.6488895482849216             True           0.5995
2   0.6767049343351716             True           0.5995
3    0.621128518027827             True           0.5990
4    0.664829316929445             True           0.5990
5    0.622062086234566             True           0.5990
6    0.608279167880712             True           0.5990
7    0.639381675975868             True           0.5990
8     1.23353765953284            False           0.5990
9   1.2598755980184615            False           0.5985
10  1.2455362990019019            False           0.5985
          linearrr__lbd linearrr__weight      preprocess  mean_test_score
0     1.666125807871538             True  MinMaxScaler()           0.5165
1    1.6427502712439634             True  MinMaxScaler()           0.5165
2   0.14439490141905037            False  MinMaxScaler()           0.5165
3    1.6842109992352

In [ ]:
print(présentation_résultat(searchLRR1,10))

In [ ]:
print(présentation_résultat(searchLRR2,2))

### Recherche encore plus fine (sert pas trop à grand chose car on va garder ce modèle)

In [153]:
hps0={"linearrr__lbd":uniform(loc=0.5,scale=1.5),"linearrr__weight":[False,True]}
searchLRR0= RandomizedSearchCV(RR,hps0,n_iter=500,scoring="accuracy",cv=5,verbose=1)
searchLRR0.fit(X_0train_forRR, Y_0train)

hps1={"linearrr__lbd":uniform(loc=1.5,scale=4.5),"linearrr__weight":[True],
     "preprocess":[MinMaxScaler()]}
searchLRR1= RandomizedSearchCV(RR,hps1,n_iter=500,scoring="accuracy",cv=5,verbose=1)
searchLRR1.fit(X_1train_forRR, Y_1train)


hps2={"linearrr__lbd":uniform(loc=0.005,scale=0.1),"linearrr__weight":[False,True],
     "preprocess":["passthrough"]}
searchLRR2= RandomizedSearchCV(RR,hps2,n_iter=500,scoring="accuracy",cv=5,verbose=1)
searchLRR2.fit(X_2train_forRR, Y_2train)


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  2.1min finished


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  2.9min finished


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:  1.5min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocess', StandardScaler()),
                                             ('linearrr', LinearRR())]),
                   n_iter=500,
                   param_distributions={'linearrr__lbd': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000258D7D182E0>,
                                        'linearrr__weight': [False, True],
                                        'preprocess': ['passthrough']},
                   scoring='accuracy', verbose=1)

### Présentation des résultats 

In [254]:
print(présentation_résultat(searchLRR0,10))


         linearrr__lbd linearrr__weight  mean_test_score
0   0.6474957713207163             True           0.5995
1   0.6710958253318303             True           0.5995
2   0.6780076073848955             True           0.5995
3    0.684706500425291             True           0.5995
4   0.6566284985873325             True           0.5995
5   0.6527012855360701             True           0.5995
6   0.6332628536368978             True           0.5995
7    0.636330197042721             True           0.5995
8   1.3388173314288454            False           0.5990
9   1.2439881721589723            False           0.5990
10  0.7009797809890661             True           0.5990
11  0.6970090412156806             True           0.5990
12  0.6959957407336215             True           0.5990
13  0.6250852638590156             True           0.5990
14  0.6983219002757041             True           0.5990
15  0.6178297739883101             True           0.5990
16  0.6158370984442172         

In [ ]:
print(présentation_résultat(searchLRR1,10))

In [ ]:
print(présentation_résultat(searchLRR2,10))

# Kernel Ridge regression 

### Crétion du kernel de manière efficiente voir ce lien pour comprendre https://stackoverflow.com/questions/47271662/what-is-the-fastest-way-to-compute-an-rbf-kernel-in-python 
Mais ce qu'il y a retenir c'est que $\| x-y \|^2=\|x\|^2 + \|y\|^2 -2x^Ty$ et d'utiliser à son avantage le broadcast de Numpy. Le ne.evaluate est là pour rendre l'opération plus rapide mais l'opération qui est faite et bien "exp(-g*(A+B-2*C))" Avec les A B C g correspondant. D'ailleurs la propriété précédente nous permet de manière générale de calculer la matrice $K(x_i,y_i)$ pour $y_1,\dots,y_k ~~x_1,\dots,x_n$  $y_i,x_i \in R^d$ La kernel matrice et un cas particulier où $n=k$ et $x_i=y_i$ pour tout $i$.
Cette fonction est contruite juste après la fonction qui construit la kernel matrice. Elle nous sera très utile en espérant que vous aviez déjà deviné pourquoi, c'est pour  calculer $\left(f(z_1),\dots,f(z_k)\right)$ oû $f(z_j)=\sum_{i=1}^{n}\alpha_i K(z_j,x_i)$ 


In [14]:
def rbf_kernel(X_train,gamma):
    X_norm = np.sum(X_train ** 2, axis = -1)
    K = ne.evaluate('exp(-g * (A + B - 2 * C))', {
        'A' : X_norm[:,None],
        'B' : X_norm[None,:],
        'C' : np.dot(X_train, X_train.T),
        'g' : gamma,
    })
    return K

In [15]:
def K_rbf_kernel(X_test,X_train,gamma):
   
    Xtt_norm = np.sum(X_test ** 2, axis = -1)
    Xtr_norm = np.sum(X_train ** 2, axis = -1)
    K = ne.evaluate('exp(-g * (A + B - 2 * C))', {
        'A' : Xtt_norm[:,None],
        'B' : Xtr_norm[None,:],
        'C' : np.dot(X_test, X_train.T),
        'g' : gamma,
    })
    return K
    

In [10]:
def addbiais(X):
    return np.hstack((X,np.ones((X.shape[0],1))))
def addzeros(X):
    n,_=X.shape
    A=np.zeros((n+1,n+1))
    A[:n,:n]=X
    return(A)

    
    


### Création du solver Kernel ridge regresion

Comme vous l'avez remarqué je laisse la possibilité d'ajouter un paramètre de biais à l'estimateur ($f(x)=\sum_{i=1}^{n}K(xi,x)\,+b)$ mais qui n'est pas pénalisé!! car cela aurait aucun sens. Cela mène bien au système d'équation que je résouds dans la partie *bais* du solver qui suit, vérifier à la main si vous n'êtes pas convaincu

In [11]:
def Kernel_Ridge_Regression(X_train,y_train,lbd,weight,gamma,biais,kernel):
    if kernel=="rbf":
        K=rbf_kernel(X_train,gamma)
    if kernel=="precomputed":
        K=X_train
    n=K.shape[0]
    w=weight
    if not(biais):
        if isinstance(weight,bool):
            A=(K+n*lbd*np.eye(n))
            alpha=solve(A,y_train,assume_a="sym")
            return alpha
        elif isinstance(weight,str):
                w1=(y_train==1).mean()
                w0=1-w1
                w=np.where(y_train==1,w1,w0)
        wi=(1/w)
        
        A=K+n*lbd*wi*np.eye(n)
        alpha=solve(A,y_train,assume_a="sym")
     
        return alpha
    else:
        
        Kb=addbiais(K)
        
        K0=addzeros(K)
    
        if isinstance(weight,bool):
            A=(Kb.T.dot(Kb)+lbd*n*K0)
            B=Kb.T.dot(y_train)
            alpha=solve(A,B,assume_a="sym")
            return alpha
        elif isinstance(weight,str):
                w1=(y_train==1).mean()
                w0=1-w1
                w=np.where(y_train==1,w1,w0)
        W=np.diag(w)
        A=(Kb.T.dot(W.dot(Kb))+lbd*n*K0)
        B=Kb.T.dot(W.dot(y_train),assume_a="sym")
        alpha=solve(A,B)
        return alpha
        
    
    

### Création de l'estimateur compatible avec l'Api de scikit 

In [12]:
class KernelRR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,weight=False,gamma=1,biais=False,kernel="rbf"):
        self.lbd=lbd
        self.weight=weight
        self.gamma=gamma
        self.biais=biais
        self.kernel=kernel
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        if isinstance(self.gamma,str):
            self.gamma=1/self.Xtr.shape[1]
        self.alpha=Kernel_Ridge_Regression(X,y,self.lbd,self.weight,self.gamma,self.biais,self.kernel)
        return self
    def decision_function(self,X):
        if self.kernel=="rbf":
            if not(self.biais):
                return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
            else:
                return addbiais(K_rbf_kernel(X,self.Xtr,self.gamma)).dot(self.alpha)
        

    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
   
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"weight":self.weight,"gamma":self.gamma,"biais":self.biais,"kernel":self.kernel}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### Recherche des hyperparamètres pour KRR avec noyau gaussien 
Pour la transformation des données ici vu qu'on va ensuite appliqué un kernel qui va rédéfinir les distance entre le points la seul transformation qui à un sens est celle où l'on standardise les données car elle conserve les rapports de distance(et donc pas affecté le kernel). De plus point très important, le paramètre *gamma* du noyau et *lambda* de la régularisation peuvent ne pas jouer de pair dans la performance de l'estimateur, il est donc difficile d'optimiser à la fois *lambda* et *gamma*. Effectivement il est possible que pour un *gamma* haut on ait besoin d'un *lambda* bas pour performer et inversement et comme on ne veut pas se perdre dans différents chemins, il semble logique de fixer *gamma* puis d'optimer *lambda*. Cela tombe bien puisque sans rentrer dans les détails mais lorsque que les données sont standardisées un paramètre de *gamma* naturel est 1/(num_features).

In [13]:
KRR=make_pipeline(StandardScaler(),KernelRR(kernel="rbf"))
print(KRR)

hps0={'kernelrr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],'kernelrr__weight':[False,"balanced"],
     'kernelrr__gamma':["auto"],"kernelrr__biais":[False,True]}
searchKRR0= GridSearchCV(KRR,hps0,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR0.fit(X_0train, Y_0train)

hps1={"kernelrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernelrr__weight":[False,"balanced"],
     "kernelrr__gamma":["auto"],"kernelrr__biais":[False,True]}
searchKRR1= GridSearchCV(KRR,hps1,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR1.fit(X_1train, Y_1train)


hps2={"kernelrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernelrr__weight":[False,"balanced"],
     "kernelrr__gamma":["auto"],"kernelrr__biais":[False,True]}
searchKRR2= GridSearchCV(KRR,hps2,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR2.fit(X_2train, Y_2train)



Pipeline(steps=[('standardscaler', StandardScaler()), ('kernelrr', KernelRR())])
Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   28.3s finished


Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   27.8s finished


Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   28.9s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('kernelrr', KernelRR())]),
             n_jobs=-1,
             param_grid={'kernelrr__biais': [False, True],
                         'kernelrr__gamma': ['auto'],
                         'kernelrr__lbd': [1e-06, 1e-05, 0.0001, 0.001, 0.01,
                                           0.1, 1, 10],
                         'kernelrr__weight': [False, 'balanced']},
             scoring='accuracy', verbose=1)

### Présentation des Résultats 
On remarque pour le training set 1 on a 10% en plus 20% en plus!!! pour le training set 2 et rien ne change pour le training set 0 par rapport au modèle linéraire. A Approfondir

In [14]:
présentation_résultat(searchKRR0,10)


,kernelrr__biais,kernelrr__gamma,kernelrr__lbd,kernelrr__weight,mean_test_score
0,True,auto,0.001,False,0.6065
1,False,auto,0.001,False,0.5995
2,False,auto,0.001,balanced,0.5945
3,False,auto,0.01,False,0.5910
4,True,auto,0.01,False,0.5910
5,False,auto,0.0001,balanced,0.5895
6,False,auto,0.0001,False,0.5885
7,True,auto,0.0001,False,0.5855
8,False,auto,1e-05,False,0.5830
9,False,auto,1e-06,False,0.5825


In [15]:
présentation_résultat(searchKRR1,10)

,kernelrr__biais,kernelrr__gamma,kernelrr__lbd,kernelrr__weight,mean_test_score
0,True,auto,0.001,False,0.6100
1,False,auto,0.001,False,0.6055
2,False,auto,0.001,balanced,0.6045
3,True,auto,0.0001,False,0.6020
4,False,auto,0.0001,balanced,0.6005
5,False,auto,0.0001,False,0.5990
6,False,auto,0.01,False,0.5915
7,True,auto,1e-05,False,0.5900
8,False,auto,1e-05,balanced,0.5880
9,False,auto,1e-05,False,0.5865


In [16]:
présentation_résultat(searchKRR2,10)

,kernelrr__biais,kernelrr__gamma,kernelrr__lbd,kernelrr__weight,mean_test_score
0,False,auto,0.001,balanced,0.7105
1,False,auto,0.001,False,0.7100
2,True,auto,0.001,False,0.7100
3,False,auto,0.0001,balanced,0.6950
4,False,auto,0.01,False,0.6950
5,True,auto,0.01,False,0.6920
6,True,auto,0.0001,False,0.6900
7,False,auto,0.0001,False,0.6895
8,False,auto,0.01,balanced,0.6830
9,False,auto,1e-05,balanced,0.6780


In [60]:
KRR=make_pipeline(StandardScaler(),KernelRR(weight=False,gamma="auto",kernel="rbf"))
print(KRR)

hps0={"kernelrr__lbd":uniform(loc=10**-4,scale=10**-2),"kernelrr__biais":[False,True]}
searchKRR0= RandomizedSearchCV(KRR,hps0,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR0.fit(X_0train, Y_0train)

hps1={"kernelrr__lbd":uniform(loc=10**-4,scale=10**-2),"kernelrr__biais":[False,True]}
searchKRR1=RandomizedSearchCV(KRR,hps1,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR1.fit(X_1train, Y_1train)


hps2={"kernelrr__lbd":uniform(loc=10**-4,scale=10**-2),"kernelrr__biais":[False,True]}
searchKRR2= RandomizedSearchCV(KRR,hps2,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR2.fit(X_2train, Y_2train)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kernelrr', KernelRR(gamma='auto'))])
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.2min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   59.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.1min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   25.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.2min finished
<ipython-input-53-f85f0f17ff12>:34: LinAlgWarning: Ill-conditioned matrix (rcond=1.69122e-35): result may not be accurate.
  alpha=solve(A,B,assume_a="sym")


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('kernelrr',
                                              KernelRR(gamma='auto'))]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'kernelrr__biais': [False, True],
                                        'kernelrr__lbd': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025BD1348280>},
                   scoring='accuracy', verbose=1)

### Présentation des résultats

In [61]:
présentation_résultat(searchKRR0,10)


,kernelrr__biais,kernelrr__lbd,mean_test_score
0,True,0.0011716461755494156,0.6045
1,True,0.0012706651107447597,0.6045
2,True,0.00345410910709065,0.6040
3,True,0.0033337409141049375,0.6040
4,False,0.0013751562280025843,0.6035
5,False,0.0027529395042676773,0.6035
6,True,0.0008560822480275876,0.6030
7,True,0.002594398421605667,0.6025
8,False,0.002846093266645726,0.6025
9,True,0.0024467352602404522,0.6025


In [62]:
présentation_résultat(searchKRR1,10)

,kernelrr__biais,kernelrr__lbd,mean_test_score
0,True,0.0016897884548361265,0.6100
1,False,0.001581817470862854,0.6100
2,True,0.0011952588239010585,0.6095
3,True,0.001676010228930762,0.6095
4,True,0.0014524948777145952,0.6095
5,False,0.001650028406301649,0.6090
6,False,0.0016310161062084872,0.6090
7,True,0.0011854223960396126,0.6085
8,False,0.0011790053423943348,0.6080
9,True,0.0024387609238354277,0.6075


In [63]:
présentation_résultat(searchKRR2,10)

,kernelrr__biais,kernelrr__lbd,mean_test_score
0,True,0.0016832940340065106,0.7130
1,False,0.001865952423818461,0.7120
2,True,0.0019369707094631227,0.7120
3,False,0.001802820603191192,0.7115
4,True,0.0015615185641109708,0.7115
5,True,0.0014168280215399187,0.7110
6,False,0.0019758158922168734,0.7110
7,True,0.002006369903099909,0.7105
8,True,0.0020918206792203873,0.7100
9,False,0.0020407581385315553,0.7100


# Logistic Regression 

In [17]:
def sigmoid(v):
    return 1/(1+np.exp(-v))
def log_loss(v):
    return np.log(1+np.exp(-v))
def cross_loss(y,v):
    return -(y*np.log(v)+(1-y)*np.log(1-v))
from scipy.special import expit

## Solver Kernel Logistic Regression 

Le critère d'arrêt que j'ai mis c'est quand la différence deux itétéres consécutifs a une norme inférieure à $\eps$. Ici aussi je laisse la possibilité d'un biais.

In [31]:
def IRLS(X_train,y_train,lbd,ga,bs,ker,n_iter,eps=10**-6,method='slow'):
    n=y_train.shape[0]
  
    if ker=="rbf":
        K=rbf_kernel(X_train,ga)
  
    #alpha=Kernel_Ridge_Regression(K,y_train,lbd,False,1,bs,"precomputed")
    #alpha=np.zeros(n)
    #l=[]
  
    if bs :
        Kb=addbiais(K)
        K0=addzeros(K)
        alpha=np.zeros(n+1)
    else:
        alpha=np.zeros(n)
    for i in range(n_iter):
   
        alpha_old=alpha
       
        if bs:
            m=Kb.dot(alpha)
            #l.append(log_loss(y_train*m).mean()+lbd*alpha[:-1].dot(K.dot(alpha[:-1])))
        
        else:
            m=K.dot(alpha)
            #l.append(log_loss(y_train*m).mean()+lbd*alpha.dot(m))
        
        
        p=sigmoid(m)
       
        weight=p*(1-p)
       
        weight=np.where(weight<0.000001,0.000001,weight)
       
   
        u=np.where(sigmoid(y_train*m)<0.000001,0.000001,sigmoid(y_train*m))
        z = m + y_train/u
    
        if not(bs):
        
            S = np.diag(weight**-1)
            A=(K+2*lbd*n*S)
            alpha=solve(A,z,assume_a="sym")
            
            #print(np.linalg.norm(alpha_old-alpha))
            
            if np.linalg.norm(alpha_old-alpha)<eps:
                break
        else:
            S = np.diag(weight)
            A=(Kb.T.dot(S.dot(Kb))+2*lbd*n*K0)
            B=Kb.T.dot(S.dot(z))
            if method=="slow":
                alpha=lstsq(A,B)[0]
            else:
                alpha=solve(A,B,assume_a="sym")
            #print(np.linalg.norm(alpha_old-alpha))
            if np.linalg.norm(alpha_old-alpha)<eps:
                break
                
       
    return alpha #,l
        
#with np.printoptions(threshold=np.inf):

### Je construit l'estimateur compatible avec l'API de scikit 

In [32]:
class KernelLR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,gamma='auto',biais=False,kernel="rbf",n_iter=15,method="slow"):
        self.lbd=lbd
        self.gamma=gamma
        self.biais=biais
        self.kernel=kernel
        self.n_iter=n_iter
        self.method=method
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        if isinstance(self.gamma,str):
            self.gamma=1/self.Xtr.shape[1]
        self.alpha=IRLS(X,y,self.lbd,self.gamma,self.biais,self.kernel,self.n_iter,method=self.method)
        return self
    def decision_function(self,X):
        if self.kernel=="rbf":
            if not(self.biais):
                return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
            else:
                return addbiais(K_rbf_kernel(X,self.Xtr,self.gamma)).dot(self.alpha)
        

    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
    def predict_proba_(self,X,y=None):
        p=sigmoid(self.decision_function(X)).reshape(-1,1)
        return hstack((p,1-p))
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"gamma":self.gamma,"biais":self.biais,"kernel":self.kernel,"n_iter":self.n_iter,
                "method":self.method}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### Recherche des Hyperparamètres 

In [33]:
KLR=make_pipeline(StandardScaler(),KernelLR(method="fast"))
print(KRR)

hps0={'kernellr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernellr__biais":[False,True]}
searchKLR0= GridSearchCV(KLR,hps0,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR0.fit(X_0train, Y_0train)

hps1={'kernellr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernellr__biais":[False,True]}
searchKLR1= GridSearchCV(KLR,hps1,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR1.fit(X_1train, Y_1train)


hps2={'kernellr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"kernellr__biais":[False,True]}
searchKLR2= GridSearchCV(KLR,hps2,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR2.fit(X_2train, Y_2train)


Pipeline(steps=[('standardscaler', StandardScaler()), ('kernelrr', KernelRR())])
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   58.5s finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  1.1min finished


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  3.0min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('kernellr', KernelLR(method='fast'))]),
             n_jobs=-1,
             param_grid={'kernellr__biais': [False, True],
                         'kernellr__lbd': [1e-06, 1e-05, 0.0001, 0.001, 0.01,
                                           0.1, 1, 10]},
             scoring='accuracy', verbose=1)

### Présentation des résultats 

In [34]:
présentation_résultat(searchKLR0,10)

,kernellr__biais,kernellr__lbd,mean_test_score
0,True,0.0001,0.6040
1,False,0.0001,0.6010
2,True,1e-05,0.5925
3,True,0.001,0.5900
4,False,1e-05,0.5890
5,False,0.001,0.5890
6,True,1e-06,0.5870
7,False,1e-06,0.5845
8,False,0.01,0.5515
9,True,0.01,0.5470


In [35]:
présentation_résultat(searchKLR1,10)

,kernellr__biais,kernellr__lbd,mean_test_score
0,True,0.0001,0.6060
1,False,0.0001,0.6055
2,True,1e-05,0.6045
3,True,1e-06,0.5975
4,False,1e-06,0.5970
5,False,1e-05,0.5965
6,False,0.001,0.5960
7,True,0.001,0.5825
8,True,0.01,0.5675
9,True,0.1,0.5655


In [36]:
présentation_résultat(searchKLR2,10)

,kernellr__biais,kernellr__lbd,mean_test_score
0,False,0.0001,0.7110
1,True,0.0001,0.7110
2,True,1e-05,0.6950
3,True,0.001,0.6940
4,False,1e-05,0.6940
5,False,0.001,0.6940
6,True,1e-06,0.6915
7,False,1e-06,0.6905
8,True,0.01,0.6705
9,False,0.01,0.6670


In [37]:
hps0={"kernellr__lbd":uniform(loc=5*10**-6,scale=5*10**-3),"kernellr__biais":[False,True]}
searchKLR0= RandomizedSearchCV(KLR,hps0,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR0.fit(X_0train, Y_0train)

hps1={"kernellr__lbd":uniform(loc=5*10**-7,scale=5*10**-3),"kernellr__biais":[False,True]}
searchKLR1=RandomizedSearchCV(KLR,hps1,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR1.fit(X_1train, Y_1train)


hps2={"kernellr__lbd":uniform(loc=5*10**-7,scale=5*10**-3),"kernellr__biais":[False,True]}
searchKLR2= RandomizedSearchCV(KLR,hps2,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR2.fit(X_2train, Y_2train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  6.2min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  5.3min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 20.4min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('kernellr',
                                              KernelLR(method='fast'))]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'kernellr__biais': [False, True],
                                        'kernellr__lbd': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000018EF0945310>},
                   scoring='accuracy', verbose=1)

In [38]:
présentation_résultat(searchKLR0,10)

,kernellr__biais,kernellr__lbd,mean_test_score
0,True,0.0001220819499826964,0.6055
1,False,0.00033872945279057935,0.6015
2,False,0.0003071798101977069,0.6000
3,True,0.00038263252712708584,0.5995
4,True,0.0002515643475037366,0.5990
5,True,0.00026197826030804064,0.5985
6,False,0.0004352733985358648,0.5980
7,True,0.00044604770346006297,0.5980
8,True,0.00044605810958857074,0.5980
9,False,0.0004736398393345459,0.5970


In [39]:
présentation_résultat(searchKLR1,10)

,kernellr__biais,kernellr__lbd,mean_test_score
0,True,0.00013217990060094894,0.6090
1,True,0.00028315839762492977,0.6080
2,False,0.00019653454392900123,0.6075
3,True,7.008659481131447e-05,0.6070
4,True,0.00030262384210140664,0.6060
5,False,6.891631624856126e-05,0.6060
6,False,0.00023360836193861098,0.6060
7,True,0.000368949313506451,0.6030
8,True,0.0003470025479657491,0.6030
9,True,0.00043110226450166384,0.6015


In [40]:
présentation_résultat(searchKLR2,10)

,kernellr__biais,kernellr__lbd,mean_test_score
0,False,9.167517204055626e-05,0.7115
1,True,7.48966790018813e-05,0.7115
2,False,0.0001255115196724855,0.7110
3,False,0.0002099223311163035,0.7105
4,False,0.00018064426304967025,0.7095
5,True,0.0002891188314545879,0.7070
6,True,0.000325347692507636,0.7045
7,True,0.0003463421706832321,0.7045
8,False,0.0003977532146016017,0.7040
9,True,0.0004530380088395331,0.7020


# SVM 
J'utilise cvxopt pour résoudre le QP

In [108]:
def SVM(X_train,y_train,C,gamma,kernel):
    n=y_train.shape[0]
    K=rbf_kernel(X_train,gamma)
    P=matrix(K,tc='d')
    q=matrix(-y_train,tc='d')
    g1=np.diag(y_train)
    G=matrix(np.vstack((g1,-g1)),tc='d')
    h=matrix(np.hstack((np.repeat(C,n),np.zeros(n))),tc='d')
    solvers.options['show_progress'] = False
    sol=solvers.qp(P,q,G,h)
    return np.array(sol['x']).reshape(-1,)

### Création de l'estimateur compatible avec l'API de scikit 

Si vous remarquez bien j'utilise tout alpha pour définir ma fonction f(x) alors que je pourrai garder seulement les alpha_i telle que alpha_i plus grand > threshold (car le solver nous fera jamais arrivé exatement à zéro quand alpha_i doit être égale à zéro) car comme on le sait une solution optimale de SVM est un vecteur sparse. Je ne l'ai pas fait car j'ai eu du mal à définir le threshold, on trouve une trace de ma tentative dans le code de l'estimateur

In [110]:
class KernelSVM(BaseEstimator,ClassifierMixin):
    def __init__(self,C=1,gamma='auto',kernel="rbf"):
        self.C=C
        self.gamma=gamma
        self.kernel=kernel
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        if isinstance(self.gamma,str):
            self.gamma=1/self.Xtr.shape[1]
        self.alpha=SVM(X,y,self.C,self.gamma,self.kernel)
        #if self.C>1:
        #    self.threshold=0.001
        #else:
        #    self.threshold=(10**-4)*self.C
        #self.Xtr=self.Xtr[self.alpha>self.threshold]
        #self.alpha=self.alpha[self.alpha>self.threshold]
        return self
    def decision_function(self,X):
        if self.kernel=="rbf":
            return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
           
    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
    def get_params(self, deep=True):
    
        return {"C": self.C,"gamma":self.gamma,"kernel":self.kernel}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [111]:
KSVM=make_pipeline(StandardScaler(),KernelSVM())
print(KSVM)

hps0={'kernelsvm__C':[10**-2,10**-1,1,10,50,100,500,1000]}
searchKSVM0= GridSearchCV(KSVM,hps0,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM0.fit(X_0train, Y_0train)

hps1={'kernelsvm__C':[10**-2,10**-1,1,10,50,100,500,1000]}
searchKSVM1= GridSearchCV(KSVM,hps1,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM1.fit(X_1train, Y_1train)


hps2={'kernelsvm__C':[10**-2,10**-1,1,10,50,100,500,1000]}
searchKSVM2= GridSearchCV(KSVM,hps2,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM2.fit(X_2train, Y_2train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kernelsvm', KernelSVM())])
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.0min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   54.3s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.0min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('kernelsvm', KernelSVM())]),
             n_jobs=-1,
             param_grid={'kernelsvm__C': [0.01, 0.1, 1, 10, 50, 100, 500,
                                          1000]},
             scoring='accuracy', verbose=1)

In [105]:
présentation_résultat(searchKSVM0,10)

,kernelsvm__C,mean_test_score
0,1,0.5940
1,50,0.5845
2,100,0.5845
3,500,0.5845
4,1000,0.5845
5,10,0.5835
6,0.1,0.5670
7,0.01,0.5225


In [106]:
présentation_résultat(searchKSVM1,10)

,kernelsvm__C,mean_test_score
0,1,0.6030
1,100,0.5865
2,1000,0.5865
3,10,0.5860
4,50,0.5860
5,500,0.5860
6,0.01,0.5250
7,0.1,0.5210


In [107]:
présentation_résultat(searchKSVM2,10)

,kernelsvm__C,mean_test_score
0,1,0.7040
1,0.1,0.6845
2,10,0.6845
3,50,0.6840
4,100,0.6840
5,500,0.6840
6,1000,0.6840
7,0.01,0.6535


In [112]:

hps0={"kernelsvm__C":uniform(loc=0.5,scale=50)}
searchKSVM0= RandomizedSearchCV(KSVM,hps0,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM0.fit(X_0train, Y_0train)

hps1={"kernelsvm__C":uniform(loc=0.5,scale=50)}
searchKSVM1= RandomizedSearchCV(KSVM,hps1,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM1.fit(X_1train, Y_1train)

hps2={"kernelsvm__C":uniform(loc=0.1,scale=10)}
searchKSVM2= RandomizedSearchCV(KSVM,hps2,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM2.fit(X_2train, Y_2train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 11.6min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   47.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 10.7min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   52.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 10.6min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('kernelsvm', KernelSVM())]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'kernelsvm__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025E77086AC0>},
                   scoring='accuracy', verbose=1)

In [114]:
présentation_résultat(searchKSVM0,10)

,kernelsvm__C,mean_test_score
0,0.5387927549952816,0.5980
1,3.217701576669379,0.5860
2,3.3052921345973814,0.5860
3,3.3229590086555234,0.5860
4,11.210320132961638,0.5850
...,...,...
79,30.21160381080791,0.5845
80,21.87280448447994,0.5845
81,44.15104435857199,0.5845
82,17.744593403843794,0.5845


In [115]:
présentation_résultat(searchKSVM1,10)

,kernelsvm__C,mean_test_score
0,0.607843559739762,0.6040
1,2.495502048004888,0.6035
2,1.6646527987647315,0.6020
3,2.651628686372881,0.6005
4,2.713293407577188,0.5990
5,3.9113988620188493,0.5910
6,3.939110050110983,0.5910
7,4.934577850759297,0.5890
8,5.1328081007472655,0.5890
9,5.247752360943725,0.5890


In [116]:
présentation_résultat(searchKSVM2,10)

,kernelsvm__C,mean_test_score
0,1.0736039527009789,0.7040
1,0.4551081390515864,0.7025
2,1.1501627523817681,0.7025
3,0.9820681609246006,0.7025
4,0.7842078691038478,0.7020
5,0.5970615635487296,0.7020
6,0.458556810377969,0.7020
7,0.8400504362283377,0.7015
8,0.15586829142405936,0.7005
9,0.3534686418418663,0.7005
